In [13]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score

In [16]:
df = pd.read_csv("diabetes.csv")
df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [18]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [19]:
# Setting random state to 620 to make the sampling reproducible
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns = 'Outcome'), df['Outcome'],
                                                    train_size = 0.8, random_state = 620)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(614, 8) (614,)
(154, 8) (154,)


In [21]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [22]:
# Define two SVC models with two different kernels
poly_svc = SVC(kernel='poly')
rbf_svc = SVC(kernel='rbf')

In [23]:
for model in [poly_svc, rbf_svc]:
    cv_score = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=5)
    print("Accuracy score for %s kernel SVC: %.3f"%(model.kernel, cv_score.mean()))

Accuracy score for poly kernel SVC: 0.735
Accuracy score for rbf kernel SVC: 0.782


In [24]:
poly_param = {'C': np.arange(0.5, 5, 0.5),
              'degree': [2, 3, 4, 5],
              'coef0': [0, 1, 2]}

poly_tune = GridSearchCV(estimator=poly_svc, param_grid=poly_param,
                         scoring='accuracy', n_jobs=-1, cv=5,
                         return_train_score=True)

poly_tune.fit(x_train, y_train)

poly_tune.best_params_

{'C': np.float64(2.5), 'coef0': 1, 'degree': 3}

In [25]:
rbf_param = {'C': np.arange(0.1, 1, 0.1),
             'gamma': np.arange(0.01, 0.1, 0.01)}

rbf_tune = GridSearchCV(estimator=rbf_svc, param_grid=rbf_param,
                        scoring='accuracy', n_jobs=-1, cv=5,
                        return_train_score=True)

rbf_tune.fit(x_train, y_train)

rbf_tune.best_params_

{'C': np.float64(0.8), 'gamma': np.float64(0.060000000000000005)}

In [26]:
poly_svc = poly_tune.best_estimator_
rbf_svc = rbf_tune.best_estimator_

In [27]:
# Creating a voting classifier using 2 optimized SVC estimators
# along with a Decision Tree classifier
voting_clf = VotingClassifier(estimators = [('poly_svc', poly_svc),
                                            ('rbf_svc', rbf_svc),
                                            ('tree', DecisionTreeClassifier())])

# Make predictions and compare the accuracy of 3 models on new data
for model in [poly_svc, rbf_svc, DecisionTreeClassifier(), voting_clf]:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("Accuracy score for %s: %.3f"%(model.__class__.__name__, accuracy_score(y_test, y_pred)))

Accuracy score for SVC: 0.721
Accuracy score for SVC: 0.740
Accuracy score for DecisionTreeClassifier: 0.721
Accuracy score for VotingClassifier: 0.760


In [29]:
bag_svc = BaggingClassifier(estimator=rbf_svc, n_estimators=100,
                            max_samples=100, bootstrap=True, n_jobs=-1)

for model in [rbf_svc, bag_svc]:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("Accuracy score for %s: %.3f"%(model.__class__.__name__, accuracy_score(y_test, y_pred)))

Accuracy score for SVC: 0.740
Accuracy score for BaggingClassifier: 0.760
